# Imports

In [51]:
# Module imports
from __future__ import print_function
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
%matplotlib inline
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import cv2
import os
import pickle
import numpy as np
from PIL import Image

In [52]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Load Data

In [53]:
rawdata = pd.read_csv('train_data.csv')
# display the top rows
rawdata.head(10)
# get a quick description of the data, including # of rows, # of features, name of each feature, type of each feature, # of non-null values
rawdata.info()
# show a summary of the numerical attributes
rawdata.describe()

rawtargetdata = pd.read_csv('train_target.csv')
# display the top rows
rawtargetdata.head(10)
# get a quick description of the data, including # of rows, # of features, name of each feature, type of each feature, # of non-null values
rawtargetdata.info()
# show a summary of the numerical attributes
rawtargetdata.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16174 entries, 0 to 16173
Columns: 2304 entries, 146 to 81.32
dtypes: int64(2304)
memory usage: 284.3 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16174 entries, 0 to 16173
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   1       16174 non-null  int64
dtypes: int64(1)
memory usage: 126.5 KB


,1
count,16174.000000
mean,1.059973
std,0.741898
min,0.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,2.000000


#  Task 1: Preprocess Data

In [54]:
# num_pipeline = Pipeline([
#     ('std_scaler', StandardScaler()),
# ])

# data_preprocessed = num_pipeline.fit_transform(rawdata)
data_preprocessed = rawdata.to_numpy()
targets_preprocessed = rawtargetdata.to_numpy()

# Task 2: Split Dataset for Training, Validation, and Testing

In [55]:
train_data, valid_data, train_target, valid_target = train_test_split(data_preprocessed, targets_preprocessed, test_size = 0.20, random_state = 42)

#print(train_data)
print(f"train_data.shape: {train_data.shape}")
print(f"train_target.shape: {train_target.shape}")
print()
print(f"valid_data.shape: {valid_data.shape}")
print(f"valid_target.shape: {valid_target.shape}")

train_data.shape: (12939, 2304)
train_target.shape: (12939, 1)

valid_data.shape: (3235, 2304)
valid_target.shape: (3235, 1)


In [56]:
class CustomDataset(Dataset):
    def __init__(self, train_data, train_target, transform = None, target_transform = None):
        super().__init__()
        self.transform = transform
        self.target_transform = target_transform
        self.data = []
        self.targets = []
        
        for image in train_data:
            reshaped_array = np.reshape(image, (48, 48))
            pil_image = Image.fromarray(np.uint8(reshaped_array))
            self.data.append(pil_image)
            rotate_pos_10 = pil_image.rotate(10)
            self.data.append(rotate_pos_10)
            rotate_neg_10 = pil_image.rotate(-10)
            self.data.append(rotate_neg_10)
            flipped = pil_image.transpose(Image.FLIP_LEFT_RIGHT)
            self.data.append(flipped)
        for target in train_target:
            self.targets.append(target[0])
            self.targets.append(target[0])
            self.targets.append(target[0])
            self.targets.append(target[0])
        #self.data = np.vstack(self.data).reshape(-1, 48, 48)
        #self.data = self.data.transpose((0, 1, 2))  # convert to Hight by Width by Channel
        
    def __getitem__(self, idx):
        img, target = self.data[idx], self.targets[idx]
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return img, target

    def __len__(self):
        return len(self.data)

In [57]:
trainingDataset = CustomDataset(train_data, train_target,
    transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.4915, 0.2470)
    ]))
validationDataset = CustomDataset(valid_data, valid_target,
    transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.4915, 0.2470)
    ]))

# print(trainingDataset.data[0])

# image, target = trainingDataset.__getitem__(0)
# plt.imshow(image)

In [58]:
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available()else {}

train_loader = torch.utils.data.DataLoader(trainingDataset, batch_size=64, shuffle=True,  **kwargs)

val_loader = torch.utils.data.DataLoader(validationDataset, batch_size=64,shuffle=False, **kwargs)

In [59]:
n_out = 3

In [66]:
model = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            #nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            #nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Flatten(), 
            nn.Linear(9216, 64),
            nn.ReLU(),
            #nn.Tanh(),
            nn.Linear(64, n_out)
)

model.to(device)

Sequential(
  (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Tanh()
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU()
  (6): Tanh()
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Flatten(start_dim=1, end_dim=-1)
  (9): Linear(in_features=9216, out_features=64, bias=True)
  (10): ReLU()
  (11): Tanh()
  (12): Linear(in_features=64, out_features=3, bias=True)
)

In [67]:
learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

n_epochs = 100

for epoch in range(n_epochs):
    model.train(True)
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        #print(labels)
        
        outputs = model(imgs)   # important:  nn.Conv2d expects a B × C × H × W shaped tensor as input
        train_loss = loss_fn(outputs, labels)
  
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
    
    model.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            
            outputs = model(imgs)
            val_loss = loss_fn(outputs, labels)
            
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())
    print("Epoch: %d, train_loss: %f, val_loss: %f, val_accuracy: %f" % (epoch, float(train_loss), float(val_loss), (correct / total)))

Epoch: 0, train_loss: 1.023329, val_loss: 1.326917, val_accuracy: 0.471561
Epoch: 1, train_loss: 1.024717, val_loss: 0.983803, val_accuracy: 0.590958
Epoch: 2, train_loss: 0.714025, val_loss: 0.975966, val_accuracy: 0.607960
Epoch: 3, train_loss: 0.607454, val_loss: 0.894923, val_accuracy: 0.617620
Epoch: 4, train_loss: 0.886874, val_loss: 0.782928, val_accuracy: 0.656569
Epoch: 5, train_loss: 0.724836, val_loss: 0.776807, val_accuracy: 0.661051
Epoch: 6, train_loss: 0.598944, val_loss: 0.761510, val_accuracy: 0.676507
Epoch: 7, train_loss: 0.730159, val_loss: 0.744661, val_accuracy: 0.689876
Epoch: 8, train_loss: 0.597894, val_loss: 0.977468, val_accuracy: 0.685162
Epoch: 9, train_loss: 0.597456, val_loss: 0.755787, val_accuracy: 0.698223
Epoch: 10, train_loss: 0.476147, val_loss: 0.692052, val_accuracy: 0.696445
Epoch: 11, train_loss: 0.722270, val_loss: 0.656553, val_accuracy: 0.709583
Epoch: 12, train_loss: 0.543271, val_loss: 0.649216, val_accuracy: 0.715456
Epoch: 13, train_loss:

KeyboardInterrupt: 